In [ ]:
import numpy as np

class Kalman2D:
    """
    Constant-velocity Kalman filter for 2D points.
    State: [px, py, vx, vy]^T
    Measurement: [px, py]^T
    """
    def __init__(self, dt=1.0, process_var=40.0, meas_var=9.0):
        self.set_dt(dt)
        self.R = np.array([[meas_var, 0.0],
                           [0.0, meas_var]], dtype=np.float64)
        self.P = np.eye(4, dtype=np.float64) * 500.0
        self.x = np.zeros((4, 1), dtype=np.float64)

    def set_dt(self, dt):
        self.dt = float(dt)
        dt2, dt3, dt4 = dt*dt, dt*dt*dt, (dt*dt)*(dt*dt)
        self.F = np.array([[1, 0, dt, 0],
                           [0, 1, 0, dt],
                           [0, 0, 1,  0],
                           [0, 0, 0,  1]], dtype=np.float64)
        # White-noise acceleration model
        q = 1.0
        self.Q = np.array([[dt4/4*q,     0,     dt3/2*q,   0     ],
                           [0,       dt4/4*q,   0,         dt3/2*q],
                           [dt3/2*q,  0,        dt2*q,     0     ],
                           [0,       dt3/2*q,   0,         dt2*q ]], dtype=np.float64)

        self.H = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0]], dtype=np.float64)

    def init_state_from_points(self, p_last, p_prev, dt):
        """Initialize [px,py,vx,vy] from last two points."""
        vx = (p_last[0] - p_prev[0]) / dt
        vy = (p_last[1] - p_prev[1]) / dt
        self.set_dt(dt)
        self.x = np.array([[p_last[0]], [p_last[1]], [vx], [vy]], dtype=np.float64)

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return float(self.x[0,0]), float(self.x[1,0])

    def update(self, z):
        z = np.array([[z[0]], [z[1]]], dtype=np.float64)
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        I = np.eye(4, dtype=np.float64)
        self.P = (I - K @ self.H) @ self.P

def kalman_predict_next(points_xy, n_future=10, dt=1.0,
                        process_var=40.0, meas_var=9.0):
    """
    points_xy: list/array of shape (N,2) with your past points (N>=2; here you have 5)
    n_future: how many future points to predict
    dt: time step between points (use seconds or frames; if you have timestamps, compute per-step dt)
    Returns:
      filtered: list of filtered estimates for the past points
      future: list of n_future predicted points (open-loop)
    """
    points_xy = np.asarray(points_xy, dtype=np.float64)
    assert points_xy.shape[0] >= 2 and points_xy.shape[1] == 2

    kf = Kalman2D(dt=dt, process_var=process_var, meas_var=meas_var)

    # Initialize using last two *earliest* points to set velocity
    kf.init_state_from_points(points_xy[1], points_xy[0], dt)

    filtered = []
    # Filter all past points (use update at each step; predict before update except for the first)
    # Step 0 already initialized at points_xy[1]; so update it then proceed
    kf.update(points_xy[1]); filtered.append((points_xy[1,0], points_xy[1,1]))
    for i in range(2, len(points_xy)):
        kf.predict()
        kf.update(points_xy[i])
        px, py = float(kf.x[0,0]), float(kf.x[1,0])
        filtered.append((px, py))

    # Now roll forward n_future steps WITHOUT updates (open-loop predictions)
    future = []
    for _ in range(n_future):
        px, py = kf.predict()
        future.append((px, py))

    return filtered, future

# ---------------------
# Example with 5 points
# ---------------------
if __name__ == "__main__":
    past5 =[(547.4562377929688, 664.9752807617188), (547.4562377929688, 664.9752807617188), (547.6067504882812, 665.0502319335938), (547.7682647705078, 665.2178344726562), (547.7899017333984, 665.1610717773438), (547.1567993164062, 665.5755920410156), (547.9822082519531, 665.3868713378906), (547.9822082519531, 665.3868713378906), (548.3366394042969, 665.3361206054688), (548.5446319580078, 665.1954345703125), (547.4718017578125, 665.8389892578125), (547.8800354003906, 665.7717895507812), (549.3073577880859, 666.0589294433594), (549.3073577880859, 666.0589294433594), (549.1675720214844, 666.2605285644531), (548.4226531982422, 666.5415954589844), (549.7260894775391, 667.0595703125), (549.5921325683594, 667.0041809082031), (551.1793975830078, 667.5372009277344), (551.1793975830078, 667.5372009277344), (552.6800384521484, 667.5756530761719), (552.1343841552734, 667.7333679199219), (551.9706726074219, 667.9080810546875), (553.7620239257812, 668.2013854980469), (552.3340301513672, 668.4747619628906), (552.3340301513672, 668.4747619628906), (553.3416900634766, 669.2948303222656), (554.0045928955078, 670.2202453613281), (553.3972473144531, 668.20654296875), (554.7648620605469, 669.9845581054688), (553.5201873779297, 668.0927734375), (553.5201873779297, 668.0927734375), (555.6385650634766, 670.3902282714844), (555.8059539794922, 670.56591796875), (555.6056976318359, 670.9396362304688), (558.015869140625, 673.0419921875), (557.9539337158203, 673.4329528808594), (557.9539337158203, 673.4329528808594), (558.2793121337891, 673.7628479003906), (557.2581634521484, 671.6558532714844), (557.4095458984375, 672.4604797363281), (558.5131530761719, 672.8908996582031), (560.0292053222656, 673.4920654296875), (560.0292053222656, 673.4920654296875), (560.5952606201172, 673.8172912597656), (560.1651611328125, 672.5535888671875), (558.4785308837891, 673.6336975097656), (560.0815277099609, 675.10693359375), (558.2805328369141, 674.9546508789062), (558.2805328369141, 674.9546508789062)]


    filtered, next10 = kalman_predict_next(past5, n_future=50, dt=1.0,
                                           process_var=40.0, meas_var=9.0)
    print("Filtered past estimates:")
    for p in filtered:
        print(p)
    print("\nPredicted next 10:")
    for p in next10:
        print(p)


Filtered past estimates:
(np.float64(547.4562377929688), np.float64(664.9752807617188))
(547.6041358614835, 665.0489299215469)
(547.7650833456435, 665.201440589444)
(547.8278474063435, 665.208172948711)
(547.4474060663385, 665.4646103649409)
(547.7237618687592, 665.4791974630223)
(547.893615025155, 665.467855803979)
(548.1849654277468, 665.4175557966979)
(548.4631783075919, 665.2991721377967)
(548.0084913173959, 665.5812813715761)
(547.9140389297569, 665.7265701721623)
(548.6577678246114, 665.9607467519264)
(549.1325539080353, 666.086613789406)
(549.3047191041776, 666.2484071904285)
(548.936911844042, 666.4791580800398)
(549.3833058045947, 666.8833524275283)
(549.5834552279763, 667.0713811008089)
(550.55615359266, 667.4361384497818)
(551.1269624058341, 667.6207110047986)
(552.2259971606885, 667.7055477086753)
(552.512213517918, 667.8020068246625)
(552.4645078732273, 667.926687318841)
(553.3287594282538, 668.1409130628833)
(553.0158223402341, 668.4014386362936)
(552.7252298168896, 668.5